In [5]:
import tensorflow as tf

In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data",one_hot = True)

#二次代价函数训练MNIST
#定义每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#定义两个placeholder
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])

#创建一个简单的神经网络
Weight = tf.Variable(tf.zeros([784,10]))
bias = tf.Variable(tf.zeros([1,10]))
prediction = tf.nn.softmax(tf.matmul(x,Weight)+bias)

#二次代价函数 loss指标表示模型的好壞
loss = tf.reduce_mean(tf.square(y-prediction))
#使用梯度下降法
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大值所在的位置
#计算准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("iter"+str(epoch)+",testing accuracy"+str(acc))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
iter0,testing accuracy0.8297
iter1,testing accuracy0.8705
iter2,testing accuracy0.8824
iter3,testing accuracy0.8882
iter4,testing accuracy0.8946
iter5,testing accuracy0.8966
iter6,testing accuracy0.8999
iter7,testing accuracy0.9016
iter8,testing accuracy0.9036
iter9,testing accuracy0.9054
iter10,testing accuracy0.9064
iter11,testing accuracy0.9074
iter12,testing accuracy0.9083
iter13,testing accuracy0.9087
iter14,testing accuracy0.9098
iter15,testing accuracy0.9109
iter16,testing accuracy0.9118
iter17,testing accuracy0.9125
iter18,testing accuracy0.9126
iter19,testing accuracy0.9133
iter20,testing accuracy0.9142


In [8]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data",one_hot = True)

#不改变激活函数，改用交叉熵代价函数，同时配合tensorboard
#定义每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#命名空间
with tf.name_scope('input'):
    #定义两个placeholder
    x=tf.placeholder(tf.float32,[None,784],name='x-input')
    y=tf.placeholder(tf.float32,[None,10],name='y-input')

with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        Weight = tf.Variable(tf.zeros([784,10]),name='w')
    with tf.name_scope('biases'):
        bias = tf.Variable(tf.zeros([1,10]),name='b')
    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(tf.matmul(x,Weight)+bias)

#交叉熵,加快模型的收敛速度
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
with tf.name_scope('train'):
    #使用梯度下降法
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #结果存放在一个布尔型列表中
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大值所在的位置
    with tf.name_scope('accuracy'):
        #计算准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("iter"+str(epoch)+",testing accuracy"+str(acc))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
iter0,testing accuracy0.8249
iter1,testing accuracy0.8923
iter2,testing accuracy0.9011
iter3,testing accuracy0.9056
iter4,testing accuracy0.9089
iter5,testing accuracy0.9101
iter6,testing accuracy0.9113
iter7,testing accuracy0.913
iter8,testing accuracy0.9144
iter9,testing accuracy0.9154
iter10,testing accuracy0.9172
iter11,testing accuracy0.9189
iter12,testing accuracy0.9183
iter13,testing accuracy0.919
iter14,testing accuracy0.9203
iter15,testing accuracy0.9199
iter16,testing accuracy0.9202
iter17,testing accuracy0.9204
iter18,testing accuracy0.9204
iter19,testing accuracy0.9212
iter20,testing accuracy0.9218


In [9]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data",one_hot = True)

#增加中间层和dropout，模拟过拟合情况
#定义每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#定义两个placeholder
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

#创建一个简单的神经网络
Weight_L1 = tf.Variable(tf.truncated_normal([784,500],stddev=0.1))
bias_L1 = tf.Variable(tf.zeros([500])+0.1)
L1 = tf.nn.tanh(tf.matmul(x,Weight_L1)+bias_L1)
L1_drop = tf.nn.dropout(L1,keep_prob)

Weight_L2 = tf.Variable(tf.truncated_normal([500,300],stddev=0.1))
bias_L2 = tf.Variable(tf.zeros([300])+0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop,Weight_L2)+bias_L2)
L2_drop = tf.nn.dropout(L2,keep_prob)

Weight_L3 = tf.Variable(tf.truncated_normal([300,100],stddev=0.1))
bias_L3 = tf.Variable(tf.zeros([100])+0.1)
L3 = tf.nn.tanh(tf.matmul(L2_drop,Weight_L3)+bias_L3)
L3_drop = tf.nn.dropout(L3,keep_prob)

Weight_L4 = tf.Variable(tf.truncated_normal([100,10],stddev=0.1))
bias_L4 = tf.Variable(tf.zeros([10])+0.1)

#输出层不在有drop_out
prediction = tf.nn.softmax(tf.matmul(L3_drop,Weight_L4)+bias_L4)

#交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用梯度下降法
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大值所在的位置
#计算准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
            
        testing_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        training_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
        print("iter"+str(epoch)+",testing accuracy"+str(testing_acc)+",training accuracy"+str(training_acc))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
iter0,testing accuracy0.9307,training accuracy0.92794544
iter1,testing accuracy0.9451,training accuracy0.9473091
iter2,testing accuracy0.9527,training accuracy0.9575091
iter3,testing accuracy0.9565,training accuracy0.96287274
iter4,testing accuracy0.9599,training accuracy0.96867275
iter5,testing accuracy0.9645,training accuracy0.9736
iter6,testing accuracy0.9662,training accuracy0.97654545
iter7,testing accuracy0.9671,training accuracy0.979
iter8,testing accuracy0.9704,training accuracy0.98138183
iter9,testing accuracy0.9698,training accuracy0.98350906
iter10,testing accuracy0.9706,training accuracy0.9846727
iter11,testing accuracy0.9724,training accuracy0.98614544
iter12,testing accuracy0.972,training accuracy0.9870909
iter13,testing accuracy0.9741,training accuracy0.98809093
iter14,testing acc

In [10]:
#定义每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#定义两个placeholder
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])

#创建一个简单的神经网络
Weight = tf.Variable(tf.zeros([784,10]))
bias = tf.Variable(tf.zeros([1,10]))
prediction = tf.nn.softmax(tf.matmul(x,Weight)+bias)

#交叉熵
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#优化器
train_step = tf.train.AdamOptimizer(0.01).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大值所在的位置
#计算准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("iter"+str(epoch)+",testing accuracy"+str(acc))

iter0,testing accuracy0.9211
iter1,testing accuracy0.9258
iter2,testing accuracy0.9291
iter3,testing accuracy0.9269
iter4,testing accuracy0.9283
iter5,testing accuracy0.9304
iter6,testing accuracy0.9258
iter7,testing accuracy0.9289
iter8,testing accuracy0.9312
iter9,testing accuracy0.9303
iter10,testing accuracy0.9317
iter11,testing accuracy0.9298
iter12,testing accuracy0.9314
iter13,testing accuracy0.9337
iter14,testing accuracy0.9318
iter15,testing accuracy0.9316
iter16,testing accuracy0.9329
iter17,testing accuracy0.9316
iter18,testing accuracy0.9333
iter19,testing accuracy0.9321
iter20,testing accuracy0.9305
